## All my life, I've been searching for something

In [28]:
from pathlib import Path
import matplotlib.pyplot as plt
import re
import numpy as np

In [142]:
pattern = 'Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)'

pattern = re.compile(pattern)

beacons = [[sx, sy, bx, by, abs(sx-bx) + abs(sy-by)]
           for sx, sy, bx, by in [[int(coord) for coord in re.search(pattern, line).groups()] for line in Path('beacons.txt').read_text().split('\n')]]

beacon_locations = [[bx, by] for sx, sy, bx, by, dist in beacons]
sensor_locations = [[sx, sy] for sx, sy, bx, by, dist in beacons]

In [143]:
target_row = 2000000

forbidden_xcoords = []

for sx, sy, bx, by, dist in beacons:
    remaining_dist = dist - abs(target_row - sy)
    if remaining_dist < 0:
        continue
    else:
        forbidden_xcoords += [x for x in range(sx-remaining_dist, sx + remaining_dist + 1) if [x, target_row] not in beacon_locations and [x, target_row] not in sensor_locations]
        
print(len(set(forbidden_xcoords)))

5403290


## Part 2: Nothing satisfies but I'm getting close

In [156]:
x_intercepts = np.array([[dist + sx + sy, -dist + sx + sy, dist + sx - sy, -dist + sx - sy] for sx, sy, bx, by, dist in beacons])

In [264]:
x, y = [0, 0]

inside = [[sx, sy, dist, dist - abs(sx-x) - abs(sy-y)] for sx, sy, bx, by, dist in beacons if dist - abs(sx-x) - abs(sy-y) >= 0]

vx, vy = [1, 1]

num_steps = [(dist + sx + sy - x - y)//2 + 1 for sx, sy, bx, by, dist in beacons if dist - abs(sx -x) - abs(sy-y) >= 0]

In [265]:
x, y = [max(num_steps)]*2

inside = [[sx, sy, dist, dist - abs(sx-x) - abs(sy-y)] for sx, sy, bx, by, dist in beacons if dist - abs(sx-x) - abs(sy-y) >= 0]

vx, vy = [1, -1]
vx, vy = [-1, 1]

num_steps = [(dist - sx + sy + x - y)//2 + 1 for sx, sy, bx, by, dist in beacons if dist - abs(sx -x) - abs(sy-y) >= 0]

In [266]:
inside, x, y

([[1003362, 1946094, 1586508, 643776]], 1312902, 1312902)

In [267]:
x, y = [x - max(num_steps), y + max(num_steps)]

inside = [[sx, sy, dist, dist - abs(sx-x) - abs(sy-y)] for sx, sy, bx, by, dist in beacons if dist - abs(sx-x) - abs(sy-y) >= 0]

In [268]:
inside, x, y

([[600131, 3861189, 1988710, 153194]], 48281, 2577523)